# File formats in forecats
---
En este notebook se explican los diferentes formato de archivos usados por los modelos de predicción climática.

> **GRIB:**
>
> Es un formato utilizado para almacenar datos meteorológicos definido por la Organización Meteorológica Mundial para almacenar y distribuir datos reticulados. Sus siglas significan: "General Regularly distributed Information in Binary form" por tanto es un archivo en formato binario para incrementar la eficiencia a la hora de ser almacenados. Estos archivos están conformados por mensajes GRIB que se concatenan para formar un archivo GRIB. Los archivos GRIB suelen tener la extensión .grib, .grb o .gb.
>
> Puede encontrar más información en la página de [ECMWF](https://confluence.ecmwf.int/display/CKB/What+are+GRIB+files+and+how+can+I+read+them#heading-Introduction) 

> **PT:**
>
> Es un formato que se asocia principalmente con `PyTorch`. Los archivos .pt en PyTorch generalmente contienen modelos entrenados, parámetros de red neuronal, pesos y otros datos relacionados con el modelo. Este formato es ampliamente utilizado para guardar y cargar modelos entrenados en PyTorch.

> **NC:**
>
> Este formato lleva el nombre de Network Common Data Form (NetCDF) es un conjunto de bibliotecas de software y formatos de datos que permiten crear, acceder y compartir datos científicos orientados a matrices. 

## GRIB to NC
---
Para cargar un .GRIB con xarray es necesario instalar `cfgrib`

In [ ]:
# !pip install cfgrib

In [17]:
import xarray as xr

In [18]:
current_dir = "C:\\Users\\gcuervo\\OneDrive - Universidad de Las Palmas de Gran Canaria\\Documents\\Doctorado\\DB\\Data_formats\\"
path_grib_file = current_dir + "download.grib"



data = xr.open_dataset(path_grib_file, engine='cfgrib')

Ignoring index file 'C:\\Users\\gcuervo\\OneDrive - Universidad de Las Palmas de Gran Canaria\\Documents\\Doctorado\\DB\\Data_formats\\download.grib.923a8.idx' incompatible with GRIB file


In [19]:
data

<xarray.Dataset>
Dimensions:        (latitude: 721, longitude: 1440)
Coordinates:
    number         int32 ...
    time           datetime64[ns] ...
    step           timedelta64[ns] ...
    isobaricInhPa  float64 ...
  * latitude       (latitude) float64 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude      (longitude) float64 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    valid_time     datetime64[ns] ...
Data variables:
    t              (latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-02-12T21:13 GRIB to CDM+CF via cfgrib-0.9.1...

Para convertir un GRIB a NC usamos el método ``.to_netcdf()``

In [20]:
data.to_netcdf(current_dir + 'netcdf_file.nc')

## NC to Pytorch
---
Desafortunadamente Pytorch no tiene métodos nativos para cargar datos tipo NC por lo que se utilizan otros paquetes.

En estos foros discuten sobre como alimentar modelos de ML con datos NetCDF 
[NC to tensorflow](https://www.noahbrenowitz.com/post/loading_netcdfs/)
[Discusión](https://discourse.pangeo.io/t/favorite-way-to-go-from-netcdf-xarray-to-torch-tf-jax-et-al/2663)

El paquete ``Xbatcher`` pare ser la herramiente más eficiente para dividir un NC en batches para alimentar dataloaders de ``Pytorch``
[Xbatcher](https://figshare.com/articles/presentation/Xbatcher_-_A_Python_Package_That_Simplifies_Feeding_Xarray_Data_Objects_to_Machine_Learning_Libraries/22264072/1)

In [ ]:
# !pip install xbatcher

In [46]:
import xbatcher
import torch

In [32]:
bgen = xbatcher.BatchGenerator(data,
                               input_dims={'latitude': 10}
                               )
# Numero de batches
len(bgen)

72

In [30]:
batch_list = [batch for batch in bgen]
batch_list

[<xarray.Dataset>
 Dimensions:        (latitude: 10, longitude: 1440)
 Coordinates:
     number         int32 0
     time           datetime64[ns] 2008-01-01T12:00:00
     step           timedelta64[ns] 00:00:00
     isobaricInhPa  float64 1e+03
   * latitude       (latitude) float64 90.0 89.75 89.5 89.25 ... 88.25 88.0 87.75
   * longitude      (longitude) float64 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
     valid_time     datetime64[ns] 2008-01-01T12:00:00
 Data variables:
     t              (latitude, longitude) float32 249.1 249.1 ... 255.2 255.2
 Attributes:
     GRIB_edition:            1
     GRIB_centre:             ecmf
     GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
     GRIB_subCentre:          0
     Conventions:             CF-1.7
     institution:             European Centre for Medium-Range Weather Forecasts
     history:                 2024-02-12T21:13 GRIB to CDM+CF via cfgrib-0.9.1...,
 <xarray.Dataset>
 Dimensions:        (latitude:

In [31]:
batch_list[2]

<xarray.Dataset>
Dimensions:        (latitude: 10, longitude: 1440)
Coordinates:
    number         int32 0
    time           datetime64[ns] 2008-01-01T12:00:00
    step           timedelta64[ns] 00:00:00
    isobaricInhPa  float64 1e+03
  * latitude       (latitude) float64 85.0 84.75 84.5 84.25 ... 83.25 83.0 82.75
  * longitude      (longitude) float64 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    valid_time     datetime64[ns] 2008-01-01T12:00:00
Data variables:
    t              (latitude, longitude) float32 260.3 260.5 ... 268.2 268.3
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-02-12T21:13 GRIB to CDM+CF via cfgrib-0.9.1...

In [44]:
dataset = xbatcher.loaders.torch.MapDataset(bgen, bgen)
dataset

In [48]:
loader = torch.utils.data.DataLoader(dataset)
loader

In [49]:
for x_batch, y_batch in loader:
    pass  # Aqui iria nuestro modelo

x_batch

tensor([[[261.1901, 261.1784, 261.1686,  ..., 261.2253, 261.2136, 261.2018],
         [260.9850, 260.9792, 260.9753,  ..., 261.0046, 260.9987, 260.9909],
         [260.8268, 260.8210, 260.8171,  ..., 260.8425, 260.8366, 260.8307],
         ...,
         [260.0866, 260.0847, 260.0807,  ..., 260.0944, 260.0925, 260.0886],
         [260.2097, 260.2097, 260.2097,  ..., 260.2116, 260.2116, 260.2097],
         [260.3600, 260.3600, 260.3600,  ..., 260.3581, 260.3581, 260.3581]]])